Imports + Auth

In [45]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

subscription_key = "74e4096a2d984a61a75e1c640ba6e25e"
endpoint = "https://ai-vision-andrei-diaconescu.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

Processing the image

In [68]:
# img = open("images/test1.png", "rb")
img = open("images/test2.jpeg", "rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!


Evaluation

In [47]:
def longest_common_subsequence(str1, str2):
  dp = [[0 for _ in range(len(str2) + 1)] for _ in range(len(str1) + 1)]
  for i in range(1, len(str1) + 1):
    for j in range(1, len(str2) + 1):
      if str1[i - 1] == str2[j - 1]:
        dp[i][j] = 1 + dp[i - 1][j - 1]
      else:
        dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

  return dp[-1][-1]

def hamming_dist(str1, str2):
  cnt = 0
  for i in range(min(len(str1), len(str2))):
    if str1[i] != str2[i]:
      cnt += 1
  return cnt

def levenshtein_dist(s1, s2):
    matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    for i in range(len(s1) + 1):
        matrix[i][0] = i
    for j in range(len(s2) + 1):
        matrix[0][j] = j

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                cost = 0
            else:
                cost = 1
            matrix[i][j] = min(matrix[i-1][j] + 1,    
                               matrix[i][j-1] + 1,
                               matrix[i-1][j-1] + cost)  

    return matrix[len(s1)][len(s2)]   

ground_truth = ["Google Cloud", "Platform"]
# groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]

lcs = longest_common_subsequence("".join(result), "".join(ground_truth))
hd = hamming_dist("".join(result), "".join(ground_truth))
ld = levenshtein_dist("".join(result), "".join(ground_truth))

print("Evaluations")
print("LCS:", lcs)
print("Hamming:", hd)
print("Levenshtein:", ld)

Evaluations
LCS: 20
Hamming: 0
Levenshtein: 0


Localisation Comparison

In [69]:
# bounding_boxes_truth = [
#     [177.0, 43.0, 414.0, 43.0, 414.0, 104.0, 177.0, 104.0],
#     [237.0, 113.0, 347.0, 113.0, 347.0, 150.0, 237.0, 150.0]]

bounding_boxes_truth = [
  [79.0, 299.0, 1333.0, 299.0, 1333.0, 462.0, 79.0, 462.0],
  [130.0, 580.0, 1045.0, 580.0, 1045.0, 722.0, 130.0, 722.0],
  [82.0, 920.0, 998.0, 920.0, 998.0, 1026.0, 82.0, 1026.0],
  [105.0, 1128.0, 1452.0, 1128.0, 1452.0, 1368.0, 105.0, 1368.0],
  ]

bound_boxes_detected = []
for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        bound_boxes_detected.append(line.bounding_box)

print("Correct bounding boxes: ")
print(bounding_boxes_truth)

print("\nDetected bounding boxes: ")
print(bound_boxes_detected)

Correct bounding boxes: 
[[79.0, 299.0, 1333.0, 299.0, 1333.0, 462.0, 79.0, 462.0], [130.0, 580.0, 1045.0, 580.0, 1045.0, 722.0, 130.0, 722.0], [82.0, 920.0, 998.0, 920.0, 998.0, 1026.0, 82.0, 1026.0], [105.0, 1128.0, 1452.0, 1128.0, 1452.0, 1368.0, 105.0, 1368.0]]

Detected bounding boxes: 
[[86.0, 314.0, 1335.0, 287.0, 1336.0, 443.0, 86.0, 478.0], [140.0, 590.0, 1045.0, 587.0, 1046.0, 723.0, 140.0, 727.0], [81.0, 915.0, 1007.0, 926.0, 1004.0, 1039.0, 78.0, 1014.0], [108.0, 1129.0, 1450.0, 1151.0, 1446.0, 1293.0, 105.0, 1259.0]]


Enhancing the text recognition

In [144]:
from PIL import Image, ImageFilter, ImageEnhance

img = Image.open('images/test1.png')

img = img.convert('L')

enhancer = ImageEnhance.Contrast(img)
img = enhancer.enhance(2.0)

threshold = 140
img = img.point(lambda p: p > threshold and 255)

img.save('images/test1_enhanced.png')

In [161]:
from PIL import Image

threshold = 170 

def is_black_or_gray(r, g, b):
    avg_intensity = (r + g + b) / 3
    if avg_intensity < threshold:
        return True
    else:
        return False
    
image = Image.open('images/test1.png')

image = image.convert('RGB')

binarized_image = Image.new('1', image.size, 255) 

for x in range(image.width):
    for y in range(image.height):
        r, g, b = image.getpixel((x, y))
        if is_black_or_gray(r, g, b):
            binarized_image.putpixel((x, y), 0) 

# Save the binarized image
binarized_image.save('images/test1_enhanced.png')

In [162]:
img = open("images/test1_enhanced.png", "rb")
# img = open("images/test2_enhanced.jpeg", "rb")

read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

Google Cloud
Platform
